# Naviagting a virtual world using Dynamic Programming

## Model-Based Reinforcement Learning (Dynamic programming algorithms)

### Dependencies

In [1]:
import sys
import gym
import numpy as np

### Helpers

In [2]:
# Calculate a state-value function
def one_step_lookahead(env, state, V, discount):
    """
    Helper function to calculate a state-value function.
    
    :param env: object
        Initialized OpenAI gym environment object
    :param V: 2-D tensor
        Matrix of size nSxnA, each cell represents 
        a probability of taking actions
    :param state: int
        Agent's state to consider
    :param discount: float
        MDP discount factor
    
    :return:
        A vector of length nA containing the expected value of each action
    """
    n_actions = env.env.nA
    action_values = np.zeros(shape=n_actions)
    for action in range(n_actions):
        for prob, next_state, reward, done in env.env.P[state][action]:
            action_values[action] += prob * (reward + discount * V[next_state])
    return action_values


# Evaluate a policy given a deterministic environment.
def policy_eval(policy, env, discount=1.0, theta=1e-9, max_iter=1e9):
    """
    Evaluate a policy given a deterministic environment.
    
    :param policy: 2-D tensor
        Matrix of size nSxnA, each cell represents 
        a probability of taking actions
    :param env: object
        Initialized OpenAI gym object
    :param discount: float default 1.0
        MDP discount factor. Float range from 0 to 1.
    :param theta: float default 1e-9
        A thereshold of value function change
    :param max_iter: int default qe-9    
        Maximum number of iteration to prevent 
        infinite loops.
    
    :return:
        A vector of size nS, which represent 
        a value function for each state.
    """
    n_states = env.env.nS
    # number of evaluation iterations
    eval_iters = 1
    # initialize a value function for each state as zeros
    V = np.zeros(shape=n_states)
    # repeat until value change is below the threshold
    for i in range(int(max_iter)):
        # init a change of value function as zero
        delta = 0
        # iterate through each state
        for state in range(n_states):
            # init a new value of current state
            v = 0
            # Try all possible actions which can be taken from this state
            for action, action_prob in enumerate(policy[state]):
                # evaluate how good each next state will be
                for state_prob, next_state, reward, done in env.env.P[state][action]:
                    # calculate the expected value
                    v += action_prob * state_prob * (reward + discount * V[next_state])
            # claculate the absolute change of value function
            delta = max(delta, np.abs(V[state] - v))
            # update value function
            V[state] = v
        eval_iters += 1
        
        # terminate if value change is insignificant
        if delta < theta:
            print(f'Policy evaluation terminated at {eval_iters} iterations.')
            return V

## Value iteration

In [3]:
def value_iteration(env, discount=1e-1, theta=1e-9, max_iter=1e4):
    """
    Value iteration algorithm to solve MDP.
    
    :param env: object
        Initaized OpenAI gym environment object
    :param discount: float default 1e-1
        MDP discount factor
    :param theta: float default 1e-9
        Stopping threshold. If the value of all states
        changes less than theta in one iteration
    :param max_iter: int
        Maximum number of iterations that can be ever performed
        (to prevent infinite horizon)
    
    :return: tuple(policy, V)
        policy: the optimal policy determined by the value function
        V: the optimal value determined by the value function
    """
    # initalized state-value function with zeros for each env state
    V = np.zeros(env.env.nS)
    
    for i in range(int(max_iter)):
        # early stopping condition
        delta = 0
        # update each state
        for state in range(env.env.nS):
            # Do a one-step lookahead to calculate state-action values
            action_value = one_step_lookahead(env, state, V, discount)
            # select best action to perform based on the highest state-action values
            best_action_value = np.max(action_value)
            # calculate change in value
            delta = max(delta, np.abs(V[state] - best_action_value))
            # update the value function for current state
            V[state] = best_action_value
            
        # check if we can stop
        if delta < theta:
            print(f'Value iteration converged at iteration #{i+1:,}')
            break
    
    # create deterministic policy using the optimal value function
    policy = np.zeros(shape=[env.env.nS, env.env.nA])
    
    for state in range(env.env.nS):
        # one step lookahead to find the best action for this state
        action_value = one_step_lookahead(env, state, V, discount)
        #select the best action based on the highest state-action value
        best_action = np.argmax(action_value)
        # update the policy to perform a better action at a current state
        policy[state, best_action] = 1.0
    
    return policy, V

## Policy iteration

In [4]:
def policy_iteration(env, discount=1.0, max_iter=1e9):
    """
    Policiy iteration algorithm to solve MDP.
    
    :param env: object
        Initaized OpenAI gym environment object
    :param discount: float default 1.0
        MDP discount factor
    :param theta: float default 1e-9
        Stopping threshold. If the value of all states
        changes less than theta in one iteration
    :param max_iter: int
        Maximum number of iterations that can be ever performed
        (to prevent infinite horizon)
        
    :return: tuple(policy, V)
        policy: the optimal policy determined by the value function
        V: the optimal value determined by the value function
    """
    # start with random policy
    # n_states x n_actions / n_actions
    n_states = env.env.nS
    n_actions = env.env.nA
    policy = np.ones(shape=[n_states, n_actions]) / n_actions
    # init counter of evaluated policies
    evaluated_policies = 1
    
    # repeat until convergence or critical number of iterations reached
    for i in range(int(max_iter)):
        stable_policy = False
        # Evaluate current policy
        V = policy_eval(policy, env, discount)
        # go through each state & try to improve actions that were taken
        for state in range(n_states):
            curr_action = np.argmax(policy[state])
            # look one step ahead and evaluate if curr_action is optimal
            # will try every possible action in a curr_state
            action_value = one_step_lookahead(env, state, V, discount)
            # select best aciton 
            best_action = np.argmax(action_value)
            # if action didn't change
            if curr_action != best_action:
                stable_policy = True
            # Greedy policy update
            policy[state] = np.eye(n_actions)[best_action]
        evaluated_policies += 1
        # if the algo converged & policy is not changing anymore
        if stable_policy:
            print(f'Found stable policy after {evaluated_policies:,} evaluations.')
            return policy, V

### Play episodes (assuming we have optimal policy)

In [5]:
def play_episodes(env, episodes, policy, max_action=100, render=False):
    """
    Play `episodes` number of episodes.
    
    :param env: object
        Initaized OpenAI gym environment object
    :param episodes: int
        Number of runs per game.
    :param policy: 2-D tensor
        Matrix of size nSxnA, each cell represents 
        a probability of taking actions. The opitmal policy 
        to be used by the agent.
    :param max_action: int default 100
        Maximum number of actions an agent can take in one 
        episode.
    :param render: boolean default False    
        Maybe render the game environment. 
        (This makes computation slower and console messy)
    
    :return: tuple(wins, total_reward, average_reward)
        wins: The total number of wins the agent has
        total_reward: The agent's total accumulated reward
        average_reward: The agent's average reward
    """
    wins = 0
    total_reward, total_action = 0, 0
    
    for episode in range(episodes):
        state = env.reset()
        done, max_a = False, 0
        while max_a < max_action:
            action = np.argmax(policy[state])
            next_state, reward, done, _ = env.step(action)
            if render:
                env.render()
            max_a += 1  # increment actions taken
            total_reward += reward  # increment reward received
            state = next_state  # set current state to next state
            # terminate if we're done and increment `wins`
            if done:
                wins += 1
                break
        
        total_action += max_a
        # Logging
        sys.stdout.write(f'\rEpisodes: {episode+1:,}\tWins: {wins:,}\t'
                         f'Total rewards: {total_reward:,}\tMax action: {max_a:,}')
        sys.stdout.flush()
    
    avg_reward = total_reward / episodes
    avg_action = total_action / episodes
    print('')
    return wins, total_reward, avg_reward, avg_action

## Environment

In [6]:
env_name = 'FrozenLake8x8-v0'
env = gym.make(env_name)

## Training loop

In [ ]:
episodes = 10000  # 10k


def train(env):
    """
    Train an agent in a given environment.
    
    :param env: object
        Initaized OpenAI gym environment object
    
    :return: None
    """
    action_mapping = {
        0: '\u2191',  # up
        1: '\u2192',  # right
        2: '\u2193',  # down
        3: '\u2190'   # left
    }
    
    solvers = [
        ('Policy Iteration', policy_iteration),
        ('Value Itearation', value_iteration)
    ]
    
    for iter_name, iter_func in solvers:
        print(115*'=')
        policy, V = iter_func(env)
        
        print(f'Final policy derived using {iter_name}:')
        print(' '.join([action_mapping[action] for action in np.argmax(policy, axis=1)]))
        
        wins, total_reward, avg_reward, avg_action = play_episodes(env, episodes, policy)
        
        print(f'{iter_name} — number of wins = {wins:,}')
        print(f'{iter_name} — average reward = {avg_reward:.2f}')
        print(f'{iter_name} — average action = {avg_action:.2f}')
    print(115*'=')

In [ ]:
train(env)

Policy evaluation terminated at 203 iterations.
Found stable policy after 2 evaluations.
Final policy derived using Policy Iteration:
← ← ← ↓ ↓ ↓ ↓ ↓ ← ← ← ← ↓ ↓ ↓ ↓ ← ← ↑ ↑ ↓ ← ↓ ↓ ← ← ← → ↑ ↑ ↓ ↓ ← ← ↑ ↑ ↓ → ← ↓ ↑ ↑ ↑ → ← ↑ ↑ ↓ ↑ ↑ → ↑ ↑ ↑ ↑ ↓ ↑ → ↑ ↑ → → → ↑
Episodes: 1,888	Wins: 1,100	Total rewards: 958.0	Max action: 590

In [ ]:
env.close()